# Ex2 - Filtering and Sorting Data

This time we are going to pull data directly from the internet.

### Step 1. Import the necessary libraries

In [1]:
import pandas as pd
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder\
                    .appName('euro')\
                    .getOrCreate()

25/05/31 22:45:19 WARN Utils: Your hostname, kevin-llanos-Type1ProductConfigId resolves to a loopback address: 127.0.1.1; using 192.168.1.92 instead (on interface wlo1)
25/05/31 22:45:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/31 22:45:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv). 

### Step 3. Assign it to a variable called euro12.

In [3]:
import requests

url = 'https://raw.githubusercontent.com/guipsamora/pandas_exercises/master/02_Filtering_%26_Sorting/Euro12/Euro_2012_stats_TEAM.csv'

get_url = requests.get(url)


with open('data.csv', 'w', encoding='UTF-8') as f:

    f.write(get_url.text)


df = pd.read_csv('data.csv', sep=',')

euro12 = spark.read.csv('data.csv', sep=',', header=True)


25/05/31 22:45:37 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Step 4. Select only the Goal column.

In [4]:
euro12.printSchema()

root
 |-- Team: string (nullable = true)
 |-- Goals: string (nullable = true)
 |-- Shots on target: string (nullable = true)
 |-- Shots off target: string (nullable = true)
 |-- Shooting Accuracy: string (nullable = true)
 |-- % Goals-to-shots: string (nullable = true)
 |-- Total shots (inc. Blocked): string (nullable = true)
 |-- Hit Woodwork: string (nullable = true)
 |-- Penalty goals: string (nullable = true)
 |-- Penalties not scored: string (nullable = true)
 |-- Headed goals: string (nullable = true)
 |-- Passes: string (nullable = true)
 |-- Passes completed: string (nullable = true)
 |-- Passing Accuracy: string (nullable = true)
 |-- Touches: string (nullable = true)
 |-- Crosses: string (nullable = true)
 |-- Dribbles: string (nullable = true)
 |-- Corners Taken: string (nullable = true)
 |-- Tackles: string (nullable = true)
 |-- Clearances: string (nullable = true)
 |-- Interceptions: string (nullable = true)
 |-- Clearances off line: string (nullable = true)
 |-- Clean Sh

In [5]:
euro12.select('Goals').show()

+-----+
|Goals|
+-----+
|    4|
|    4|
|    4|
|    5|
|    3|
|   10|
|    5|
|    6|
|    2|
|    2|
|    6|
|    1|
|    5|
|   12|
|    5|
|    2|
+-----+



### Step 5. How many team participated in the Euro2012?

In [6]:
from pyspark.sql.functions import countDistinct

euro12.agg(countDistinct('Team').alias('Total_equipos')).show()

+-------------+
|Total_equipos|
+-------------+
|           16|
+-------------+



In [7]:
df['Team'].nunique()

16

### Step 6. What is the number of columns in the dataset?

In [8]:
n_cols = len(euro12.columns)

print('El numero total de columnas es de %d'%n_cols)

El numero total de columnas es de 35


In [9]:
print(f'El numero total de columnas es de {df.shape[1]}')

El numero total de columnas es de 35


### Step 7. View only the columns Team, Yellow Cards and Red Cards and assign them to a dataframe called discipline

In [10]:
discipline = euro12.select('Team', 'Yellow Cards', 'Red Cards')

discipline.show()

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|            Croatia|           9|        0|
|     Czech Republic|           7|        0|
|            Denmark|           4|        0|
|            England|           5|        0|
|             France|           6|        0|
|            Germany|           4|        0|
|             Greece|           9|        1|
|              Italy|          16|        0|
|        Netherlands|           5|        0|
|             Poland|           7|        1|
|           Portugal|          12|        0|
|Republic of Ireland|           6|        1|
|             Russia|           6|        0|
|              Spain|          11|        0|
|             Sweden|           7|        0|
|            Ukraine|           5|        0|
+-------------------+------------+---------+



In [11]:
discipline_df = df[['Team', 'Yellow Cards', 'Red Cards']]
discipline_df

,Team,Yellow Cards,Red Cards
0,Croatia,9,0
1,Czech Republic,7,0
2,Denmark,4,0
3,England,5,0
4,France,6,0
5,Germany,4,0
6,Greece,9,1
7,Italy,16,0
8,Netherlands,5,0
9,Poland,7,1


### Step 8. Sort the teams by Red Cards, then to Yellow Cards

In [12]:
discipline.orderBy(['Yellow Cards', 'Red Cards'], ascending=[False, False]).show()

+-------------------+------------+---------+
|               Team|Yellow Cards|Red Cards|
+-------------------+------------+---------+
|             Greece|           9|        1|
|            Croatia|           9|        0|
|             Poland|           7|        1|
|     Czech Republic|           7|        0|
|             Sweden|           7|        0|
|Republic of Ireland|           6|        1|
|             France|           6|        0|
|             Russia|           6|        0|
|            England|           5|        0|
|        Netherlands|           5|        0|
|            Ukraine|           5|        0|
|            Denmark|           4|        0|
|            Germany|           4|        0|
|              Italy|          16|        0|
|           Portugal|          12|        0|
|              Spain|          11|        0|
+-------------------+------------+---------+



In [13]:
discipline_df.sort_values(['Yellow Cards', 'Red Cards'], ascending=[False, False])

,Team,Yellow Cards,Red Cards
7,Italy,16,0
10,Portugal,12,0
13,Spain,11,0
6,Greece,9,1
0,Croatia,9,0
9,Poland,7,1
1,Czech Republic,7,0
14,Sweden,7,0
11,Republic of Ireland,6,1
4,France,6,0


### Step 9. Calculate the mean Yellow Cards given per Team

In [16]:
from pyspark.sql.functions import col, avg
discipline.groupBy(col('Team')).agg(avg('Yellow Cards').alias('promedio_tarjetas_amarillas')).orderBy('promedio_tarjetas_amarillas').show()

+-------------------+---------------------------+
|               Team|promedio_tarjetas_amarillas|
+-------------------+---------------------------+
|            Germany|                        4.0|
|            Denmark|                        4.0|
|            Ukraine|                        5.0|
|            England|                        5.0|
|        Netherlands|                        5.0|
|             Russia|                        6.0|
|             France|                        6.0|
|Republic of Ireland|                        6.0|
|             Sweden|                        7.0|
|     Czech Republic|                        7.0|
|             Poland|                        7.0|
|             Greece|                        9.0|
|            Croatia|                        9.0|
|              Spain|                       11.0|
|           Portugal|                       12.0|
|              Italy|                       16.0|
+-------------------+---------------------------+


In [29]:
discipline_df.groupby('Team', as_index=False).agg(avg_amarillas= ('Yellow Cards','mean')).sort_values(by='avg_amarillas')

,Team,avg_amarillas
2,Denmark,4.0
5,Germany,4.0
3,England,5.0
8,Netherlands,5.0
15,Ukraine,5.0
12,Russia,6.0
11,Republic of Ireland,6.0
4,France,6.0
14,Sweden,7.0
9,Poland,7.0


### Step 10. Filter teams that scored more than 6 goals

In [32]:
euro12.filter(col('Goals')>6).show()

25/05/31 22:56:07 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----+---------------+----------------+-----------------+----------------+--------------------------+------------+-------------+--------------------+------------+------+----------------+----------------+-------+-------+--------+-------------+-------+----------+-------------+-------------------+------------+------+--------------+----------+--------------------+---------+--------------+--------+------------+---------+-------+--------+------------+
|   Team|Goals|Shots on target|Shots off target|Shooting Accuracy|% Goals-to-shots|Total shots (inc. Blocked)|Hit Woodwork|Penalty goals|Penalties not scored|Headed goals|Passes|Passes completed|Passing Accuracy|Touches|Crosses|Dribbles|Corners Taken|Tackles|Clearances|Interceptions|Clearances off line|Clean Sheets|Blocks|Goals conceded|Saves made|Saves-to-shots ratio|Fouls Won|Fouls Conceded|Offsides|Yellow Cards|Red Cards|Subs on|Subs off|Players Used|
+-------+-----+---------------+----------------+-----------------+----------------

In [36]:
df.loc[df['Goals']>6, :]

,Team,Goals,Shots on target,Shots off target,Shooting Accuracy,% Goals-to-shots,Total shots (inc. Blocked),Hit Woodwork,Penalty goals,Penalties not scored,...,Saves made,Saves-to-shots ratio,Fouls Won,Fouls Conceded,Offsides,Yellow Cards,Red Cards,Subs on,Subs off,Players Used
5,Germany,10,32,32,47.8%,15.6%,80,2,1,0,...,10,62.6%,63,49,12,4,0,15,15,17
13,Spain,12,42,33,55.9%,16.0%,100,0,1,0,...,15,93.8%,102,83,19,11,0,17,17,18


### Step 11. Select the teams that start with G

In [37]:
euro12.filter(col('Team').startswith('G')).show()

+-------+-----+---------------+----------------+-----------------+----------------+--------------------------+------------+-------------+--------------------+------------+------+----------------+----------------+-------+-------+--------+-------------+-------+----------+-------------+-------------------+------------+------+--------------+----------+--------------------+---------+--------------+--------+------------+---------+-------+--------+------------+
|   Team|Goals|Shots on target|Shots off target|Shooting Accuracy|% Goals-to-shots|Total shots (inc. Blocked)|Hit Woodwork|Penalty goals|Penalties not scored|Headed goals|Passes|Passes completed|Passing Accuracy|Touches|Crosses|Dribbles|Corners Taken|Tackles|Clearances|Interceptions|Clearances off line|Clean Sheets|Blocks|Goals conceded|Saves made|Saves-to-shots ratio|Fouls Won|Fouls Conceded|Offsides|Yellow Cards|Red Cards|Subs on|Subs off|Players Used|
+-------+-----+---------------+----------------+-----------------+----------------

In [40]:
from pyspark.sql.functions import lower
euro12.filter(lower(col('Team')).startswith('g')).show()

+-------+-----+---------------+----------------+-----------------+----------------+--------------------------+------------+-------------+--------------------+------------+------+----------------+----------------+-------+-------+--------+-------------+-------+----------+-------------+-------------------+------------+------+--------------+----------+--------------------+---------+--------------+--------+------------+---------+-------+--------+------------+
|   Team|Goals|Shots on target|Shots off target|Shooting Accuracy|% Goals-to-shots|Total shots (inc. Blocked)|Hit Woodwork|Penalty goals|Penalties not scored|Headed goals|Passes|Passes completed|Passing Accuracy|Touches|Crosses|Dribbles|Corners Taken|Tackles|Clearances|Interceptions|Clearances off line|Clean Sheets|Blocks|Goals conceded|Saves made|Saves-to-shots ratio|Fouls Won|Fouls Conceded|Offsides|Yellow Cards|Red Cards|Subs on|Subs off|Players Used|
+-------+-----+---------------+----------------+-----------------+----------------

In [41]:
df[df['Team'].str.lower().str.startswith('g')]

,Team,Goals,Shots on target,Shots off target,Shooting Accuracy,% Goals-to-shots,Total shots (inc. Blocked),Hit Woodwork,Penalty goals,Penalties not scored,...,Saves made,Saves-to-shots ratio,Fouls Won,Fouls Conceded,Offsides,Yellow Cards,Red Cards,Subs on,Subs off,Players Used
5,Germany,10,32,32,47.8%,15.6%,80,2,1,0,...,10,62.6%,63,49,12,4,0,15,15,17
6,Greece,5,8,18,30.7%,19.2%,32,1,1,1,...,13,65.1%,67,48,12,9,1,12,12,20


### Step 12. Select the first 7 columns

In [ ]:


for column_name in euro12.columns:
    new_column_name = (
        column_name.replace(" ", "_") 
        .replace("(", "") 
        .replace(")", "")
        .replace("%", "Pct") 
        .replace(".", "")
        .replace("`", "") 
        .replace("-", "_") 
    )

    if column_name != new_column_name:
        euro12 = euro12.withColumnRenamed(column_name, new_column_name)

columnas = euro12.columns


euro12.select(*columnas[:7]).show()

+-------------------+-----+---------------+----------------+-----------------+------------------+-----------------------+
|               Team|Goals|Shots_on_target|Shots_off_target|Shooting_Accuracy|Pct_Goals_to_shots|Total_shots_inc_Blocked|
+-------------------+-----+---------------+----------------+-----------------+------------------+-----------------------+
|            Croatia|    4|             13|              12|            51.9%|             16.0%|                     32|
|     Czech Republic|    4|             13|              18|            41.9%|             12.9%|                     39|
|            Denmark|    4|             10|              10|            50.0%|             20.0%|                     27|
|            England|    5|             11|              18|            50.0%|             17.2%|                     40|
|             France|    3|             22|              24|            37.9%|              6.5%|                     65|
|            Germany|   

In [60]:
df.iloc[:, :7]

,Team,Goals,Shots on target,Shots off target,Shooting Accuracy,% Goals-to-shots,Total shots (inc. Blocked)
0,Croatia,4,13,12,51.9%,16.0%,32
1,Czech Republic,4,13,18,41.9%,12.9%,39
2,Denmark,4,10,10,50.0%,20.0%,27
3,England,5,11,18,50.0%,17.2%,40
4,France,3,22,24,37.9%,6.5%,65
5,Germany,10,32,32,47.8%,15.6%,80
6,Greece,5,8,18,30.7%,19.2%,32
7,Italy,6,34,45,43.0%,7.5%,110
8,Netherlands,2,12,36,25.0%,4.1%,60
9,Poland,2,15,23,39.4%,5.2%,48


### Step 13. Select all columns except the last 3.

In [62]:
euro12.select(*columnas[:-3]).show()

+-------------------+-----+---------------+----------------+-----------------+------------------+-----------------------+------------+-------------+--------------------+------------+------+----------------+----------------+-------+-------+--------+-------------+-------+----------+-------------+-------------------+------------+------+--------------+----------+--------------------+---------+--------------+--------+------------+---------+
|               Team|Goals|Shots_on_target|Shots_off_target|Shooting_Accuracy|Pct_Goals_to_shots|Total_shots_inc_Blocked|Hit_Woodwork|Penalty_goals|Penalties_not_scored|Headed_goals|Passes|Passes_completed|Passing_Accuracy|Touches|Crosses|Dribbles|Corners_Taken|Tackles|Clearances|Interceptions|Clearances_off_line|Clean_Sheets|Blocks|Goals_conceded|Saves_made|Saves_to_shots_ratio|Fouls_Won|Fouls_Conceded|Offsides|Yellow_Cards|Red_Cards|
+-------------------+-----+---------------+----------------+-----------------+------------------+-----------------------

In [63]:
df.iloc[:, :-3]

,Team,Goals,Shots on target,Shots off target,Shooting Accuracy,% Goals-to-shots,Total shots (inc. Blocked),Hit Woodwork,Penalty goals,Penalties not scored,...,Clean Sheets,Blocks,Goals conceded,Saves made,Saves-to-shots ratio,Fouls Won,Fouls Conceded,Offsides,Yellow Cards,Red Cards
0,Croatia,4,13,12,51.9%,16.0%,32,0,0,0,...,0,10,3,13,81.3%,41,62,2,9,0
1,Czech Republic,4,13,18,41.9%,12.9%,39,0,0,0,...,1,10,6,9,60.1%,53,73,8,7,0
2,Denmark,4,10,10,50.0%,20.0%,27,1,0,0,...,1,10,5,10,66.7%,25,38,8,4,0
3,England,5,11,18,50.0%,17.2%,40,0,0,0,...,2,29,3,22,88.1%,43,45,6,5,0
4,France,3,22,24,37.9%,6.5%,65,1,0,0,...,1,7,5,6,54.6%,36,51,5,6,0
5,Germany,10,32,32,47.8%,15.6%,80,2,1,0,...,1,11,6,10,62.6%,63,49,12,4,0
6,Greece,5,8,18,30.7%,19.2%,32,1,1,1,...,1,23,7,13,65.1%,67,48,12,9,1
7,Italy,6,34,45,43.0%,7.5%,110,2,0,0,...,2,18,7,20,74.1%,101,89,16,16,0
8,Netherlands,2,12,36,25.0%,4.1%,60,2,0,0,...,0,9,5,12,70.6%,35,30,3,5,0
9,Poland,2,15,23,39.4%,5.2%,48,0,0,0,...,0,8,3,6,66.7%,48,56,3,7,1


### Step 14. Present only the Shooting Accuracy from England, Italy and Russia

In [70]:
euro12.filter(col('Team').isin(['England', 'Italy', 'Russia'])).select(['Team', 'Shooting_Accuracy']).show()

+-------+-----------------+
|   Team|Shooting_Accuracy|
+-------+-----------------+
|England|            50.0%|
|  Italy|            43.0%|
| Russia|            22.5%|
+-------+-----------------+



In [77]:
df[df['Team'].isin(['England', 'Italy', 'Russia'])][['Team','Shooting Accuracy']]

,Team,Shooting Accuracy
3,England,50.0%
7,Italy,43.0%
12,Russia,22.5%


25/06/01 18:22:46 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 12415780 ms exceeds timeout 120000 ms
25/06/01 18:22:46 WARN SparkContext: Killing executors is not supported by current scheduler.
25/06/01 18:22:46 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at